In [15]:
from wiki_location import q2cc
WIKI_DIR = '/backup/wikidata'
api_key = open('airtable_key.txt').read()
from pyairtable import Table, Api
import sys
sys.path.insert(0, '/projekti/nelma')
from mondoDB.referencedb import provi
from collections import defaultdict, Counter

if True:
    lang2cc = defaultdict(Counter)
    for k, v in provi.items():
        lang, s, c = v['id'].split('_')
        bod = float(v['Percent']) if 'Percent' in v else 0
        if 'Status' in v:
            if v['Status'] == 'official':
                bod += 1
            elif v['Status'] == 'official_regional':
                bod += 0.5
            elif v['Status'] == 'de_facto_official':
                bod += 0.9
            elif v['Status'] == 'romanized':
                bod += 0.2
            elif v['Status'] == 'foreign':
                bod += -0.5
        if bod>1.1:
            lang2cc[f'{lang}'][c] = max(bod, lang2cc[f'{lang}'][c])
    api = Api(api_key)
    w2iso = {t['fields']['WMF']: t['fields']['qid'] if 'qid' in t['fields'] else None for t in Table(api_key, 'appUZvAm9EHZgC1Eg', 'wiki_lang').all()}
    wiki2cc = defaultdict(Counter)
    for k, v in w2iso.items():
        if len(k) == 2:
            wiki2cc[v].update(lang2cc[k])
        elif k.count('-') == 1:
            a, b = k.split('-')
            if len(a) == 2 and len(b) == 2:
                wiki2cc[v][b.upper()] += 5 
            elif len(a) == 2 and len(b) == 4:
                #wiki2cc[v].update(lang2cc[f'{k}_{b.uppser()}'])
                wiki2cc[v].update(lang2cc[a])
    wikil2cc = {k: {cc: round(100*v2/max(v.values())) for cc, v2 in v.items()} for k, v in wiki2cc.items() if len(v)>0 and max(v.values())>0}

cc2lang = defaultdict(set)
for lang, ccs in lang2cc.items():
    for cc in ccs:
        if ccs[cc]>1.4:
            cc2lang[cc].add(lang)
iso2w = {k:v for v, k in w2iso.items()}

q2cc['Q95'], q2cc['Q31'], wikil2cc['Q1860']['US'], cc2lang['CH']

('US', 'BE', 93, {'de'})

In [ ]:
import json
from tqdm.auto import tqdm
prvo = True
fo = open('/backup/wikidata/sehr.csv', 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_618_668):
    j = json.loads(l)
    d = {'qid': j['id'], 'name_hr': j['l']['hr'] if 'hr' in j['l'] else '', 'name_se': j['l']['se'] if 'se' in j['l'] else '', 
         'name_en': j['l']['en'] if 'en' in j['l'] else '', 'desc_en': j['desc']['en'] if 'en' in j['desc'] else '',
         'desc_sv': j['desc']['en'] if 'en' in j['desc'] else '',
        }
    se = False
    hr = False
    for k in ['country residence', 'birth_place', 'place of death burial', 'educated at', 'work location', 'employer', 'language']:
        ccs = set()
        for v in j[k]:
            v = v.replace('WIKI_','')
            if k != 'language':
                if v in q2cc:
                    ccs.add(q2cc[v])
            else:
                if v in wikil2cc:
                    ccs.update(wikil2cc[v])
        if 'HR' in ccs:
            hr = True
        if 'SE' in ccs:
            se = True
        d[k] = list(ccs)
    d['hrwiki'] = 'hrwiki' in j['sitelinks']
    d['swwiki'] = 'svwiki' in j['sitelinks']
    if (se and hr) or (se and d['hrwiki'] and not d['swwiki']) or (hr and not d['hrwiki'] and d['swwiki']):
        if prvo:
            fo.write('\t'.join(d.keys()))
            fo.write('\n')
            prvo = False
        fo.write('\t'.join([str(a) for a in d.values()]))
        fo.write('\n')
fo.close()

  0%|          | 0/10618668 [00:00<?, ?it/s]

In [1]:
import json
from tqdm.auto import tqdm
prvo = True
fo = open('/backup/wikidata/ushr.csv', 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_618_668):
    j = json.loads(l)
    d = {'qid': j['id'], 'name_hr': j['l']['hr'] if 'hr' in j['l'] else '', 
         'name_en': j['l']['en'] if 'en' in j['l'] else '', 'desc_en': j['desc']['en'] if 'en' in j['desc'] else '',
         'desc_sv': j['desc']['en'] if 'en' in j['desc'] else '',
        }
    se = False
    hr = False
    for k in ['country residence', 'birth_place', 'place of death burial', 'educated at', 'work location', 'employer', 'language']:
        ccs = set()
        for v in j[k]:
            v = v.replace('WIKI_','')
            if k != 'language':
                if v in q2cc:
                    ccs.add(q2cc[v])
            else:
                if v in wikil2cc:
                    ccs.update(wikil2cc[v])
        if 'HR' in ccs:
            hr = True
        if 'US' in ccs:
            se = True
        d[k] = list(ccs)
    d['hrwiki'] = 'hrwiki' in j['sitelinks']
    d['uswiki'] = 'svwiki' in j['sitelinks']
    if (se and hr) or (se and d['hrwiki'] and not d['swwiki']) or (hr and not d['hrwiki'] and d['swwiki']):
        if prvo:
            fo.write('\t'.join(d.keys()))
            fo.write('\n')
            prvo = False
        fo.write('\t'.join([str(a) for a in d.values()]))
        fo.write('\n')
    break
fo.close()

FileNotFoundError: [Errno 2] No such file or directory: '/backup/wikidata/wiki_person.jsonl'

In [ ]:
import json
from tqdm.auto import tqdm
prvo = True
fo = open('/backup/wikidata/lvhr.csv', 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_618_668):
    j = json.loads(l)
    d = {'qid': j['id'], 'name_hr': j['l']['hr'] if 'hr' in j['l'] else '', 
         'name_en': j['l']['en'] if 'en' in j['l'] else '', 'desc_en': j['desc']['en'] if 'en' in j['desc'] else '',
         'desc_sv': j['desc']['en'] if 'en' in j['desc'] else '',
        }
    se = False
    hr = False
    for k in ['country residence', 'birth_place', 'place of death burial', 'educated at', 'work location', 'employer', 'language']:
        ccs = set()
        for v in j[k]:
            v = v.replace('WIKI_','')
            if k != 'language':
                if v in q2cc:
                    ccs.add(q2cc[v])
            else:
                if v in wikil2cc:
                    ccs.update(wikil2cc[v])
        if 'HR' in ccs:
            hr = True
        if 'US' in ccs:
            se = True
        d[k] = list(ccs)
    d['hrwiki'] = 'hrwiki' in j['sitelinks']
    d['lvwiki'] = 'lvwiki' in j['sitelinks']
    if (se and hr) or (se and d['hrwiki'] and not d['lvwiki']) or (hr and not d['lvwiki'] and d['lvwiki']):
        if prvo:
            fo.write('\t'.join(d.keys()))
            fo.write('\n')
            prvo = False
        fo.write('\t'.join([str(a) for a in d.values()]))
        fo.write('\n')
fo.close()